Sentiment analysis on tweets for various companies, want to see how they're changing with time, and whether or not this is a good idea to choose where to throw $$

Still need to confirm tweets are organized by date, that will be done tmrw


The majority of this code is shamelessly stolen from:
https://github.com/ciurana2016/predict_stock_py/blob/master/predict_stock.py

AND 

https://github.com/Avhirup/Stock-Market-Prediction-Challenge/blob/master/Predicting%20Stock%20Prices%20Challenge.ipynb

Thanks!

Update: It appears there is an issue with downloading the stocks from NASDAQ, will check this out later. 


In [ ]:
import os
import sys
import tweepy
import requests
import numpy as np
import pandas as pd

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from textblob import TextBlob

In [ ]:
#This I would prefer to grab from the web, continuously update
#in a more convenient manner. 
data = pd.read_csv('SNAP.csv')

#This doesn't work for now, let's just do it manually. 


Plan: 

Make a DNN, doesn't need to be recursive

Use tweets to bolster if they line up and are positive, idk

In [ ]:
del data['High'], data['Low'], data['Adj Close'], data['Volume']


The openining price of one day is the input, and what it opens as the next day is the output 


In [ ]:
dat = data.values

In [ ]:
dat[0,2]

In [ ]:
X = []
Y =[]
for p in range(len(dat)):
    X.append([dat[p,1]])   #,dat[p,2]])
    
for q in range(len(dat)-1):
    Y.append(dat[q+1,1])
del X[-1]

X = np.array(X)
Y = np.array(Y)

In [ ]:
model = Sequential()
model.add(Dense(8, input_dim=1, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, Y, nb_epoch=200, batch_size=2, verbose=1)


In [ ]:
print(X,Y)

In [ ]:
Y

In [ ]:
def makenet(pricesnshit):
    X = []
    for i in range(len(pricesnshit)):
        X.append(pricesnshit[i,1],pricesnshit(i,2))
    return X

In [ ]:
type(data.values)

In [ ]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
user = tweepy.API(auth)


In [ ]:
def stock_sentiment(quote, num_tweets):
    # Checks if the sentiment for our quote is
    # positive or negative, returns True if
    # majority of valid tweets have positive sentiment
    list_of_tweets = user.search(quote, count=num_tweets)
    positive, null = 0, 0

    for tweet in list_of_tweets:
       # print(tweet.text)
        #print("")
        blob = TextBlob(tweet.text).sentiment
        if blob.subjectivity == 0:
            null += 1
            next
        if blob.polarity > 0:
            positive += 1

    if positive > ((num_tweets - null)/2):
        #if over half the tweets are considered positive polarity, this is good! 
        return True

In [ ]:
def get_historical(quote):
    # Download file from google finance
    url = 'http://www.google.com/finance/historical?q=NASDAQ%3A'+quote+'&output=csv'
    r = requests.get(url, stream=True)

    if r.status_code != 400:
        with open(FILE_NAME, 'wb') as f:
            for chunk in r:
                f.write(chunk)

        return True



In [ ]:
def stock_prediction():

    # Collect data points from csv
    dataset = []

    with open(FILE_NAME) as f:
        for n, line in enumerate(f):
            if n != 0:
                dataset.append(float(line.split(',')[1]))

    dataset = np.array(dataset)

    # Create dataset matrix (X=t and Y=t+1)
    def create_dataset(dataset):
        dataX = [dataset[n+1] for n in range(len(dataset)-2)]
        return np.array(dataX), dataset[2:]
        
    trainX, trainY = create_dataset(dataset)

    # Create and fit Multilinear Perceptron model
    model = Sequential()
    model.add(Dense(8, input_dim=1, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, nb_epoch=200, batch_size=2, verbose=1)

    # Our prediction for tomorrow
    prediction = model.predict(np.array([dataset[0]]))
    result = 'The price will move from %s to %s' % (dataset[0], prediction[0][0])

    return result

In [ ]:
stock_quote = input('Enter a stock quote from NASDAQ (e.j: AAPL, FB, GOOGL): ').upper()

# Check if the stock sentiment is positve
if not stock_sentiment(stock_quote, num_tweets=20):
    print('This stock has bad sentiment, please re-run the script')
    sys.exit()

# Check if we got te historical data
if not get_historical(stock_quote):
    print('Google returned a 404, please re-run the script and')
    print('enter a valid stock quote from NASDAQ')
    sys.exit()

# We have our file so we create the neural net and get the prediction
print(stock_prediction())

# We are done so we delete the csv file
os.remove(FILE_NAME)

In [ ]:
stock_sentiment('fit',20)